In [35]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

from sklearn.preprocessing import StandardScaler

import joblib

import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
# Charger le nouveau fichier CSV
new_data = pd.read_csv("./billets_production.csv")
display(new_data.head())
display(new_data.isnull().sum())

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5


diagonal        0
height_left     0
height_right    0
margin_low      0
margin_up       0
length          0
id              0
dtype: int64

In [37]:
# Sélectionner les colonnes géométriques (sans la colonne cible)
X_new = new_data[['length', 'height_left', 'height_right', 'margin_up', 'margin_low', 'diagonal']]

In [38]:
# Charger le scaler et les colonnes
scaler, columns = joblib.load('./scaler.pkl')

# Assurer que X_new est un DataFrame avec les mêmes colonnes
X_new = pd.DataFrame(X_new, columns=columns)

# Réorganiser les colonnes dans le même ordre que celles enregistrées
X_new = X_new[columns]

# Appliquer le scaler
X_new_scaled = scaler.transform(X_new)

In [39]:
# Charger le modèle
rf_model = joblib.load('best_rf_model.pkl')

# Charger le seuil optimal
with open('optimal_threshold_rf.txt', 'r') as f:
    loaded_threshold_rf = float(f.read())

print("Modèle et seuil optimal chargés avec succès.")

Modèle et seuil optimal chargés avec succès.


In [40]:
# Prédire les probabilités pour les nouvelles données avec la régression logistique
y_proba_new = rf_model.predict_proba(X_new_scaled)[:, 1]

# Appliquer le seuil chargé pour obtenir les prédictions finales pour les nouvelles données
y_pred_new = (y_proba_new >= loaded_threshold_rf)

# Ajouter la prédiction ajustée au dataframe original pour visualiser
new_data['Prediction_is_genuine'] = y_pred_new

# Afficher les résultats
display(new_data[['length', 'height_left', 'height_right', 'margin_up', 'margin_low', 'diagonal', 'Prediction_is_genuine']].head())

,length,height_left,height_right,margin_up,margin_low,diagonal,Prediction_is_genuine
0,111.42,104.01,103.54,3.30,5.21,171.76,False
1,112.09,104.17,104.13,3.31,6.00,171.87,False
2,111.57,104.58,104.29,3.39,4.99,172.00,False
3,113.20,104.55,104.34,3.03,4.44,172.49,True
4,113.33,103.63,103.56,3.16,3.77,171.65,True


In [41]:
# Sauvegarder le fichier avec les prédictions
new_data.to_csv('billets_avec_predictions_RF.csv', index=False)

In [42]:
# Charger le nouveau fichier CSV
new_data_prediction = pd.read_csv("./billets_avec_predictions_RF.csv")

# Vérifier les premières lignes pour s'assurer que le fichier est bien formaté
display(new_data_prediction.head())

,diagonal,height_left,height_right,margin_low,margin_up,length,id,Prediction_is_genuine
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1,False
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2,False
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3,False
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4,True
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5,True
